In [ ]:
# !pip install textblob

# !pip install tweepy

# !pip install pycountry

# !pip install wordcloud 

# !pip install langdetect

In [1]:
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [18]:
from datetime import timedelta

In [ ]:
# import config
# # put ur keys in a py file as a dictionary and import them so u can keep em secret
# consumer_key = config.params['consumer_key'] #api key 
# consumer_secret = config.params['consumer_secret'] #api secret 
# access_token = config.params['access_token'] #access token  
# access_token_secret = config.params['access_token_secret'] #access secret  
# bear = config.params['bear'] #bearer token

In [2]:
from keys import *

In [4]:
client = tweepy.Client(bearer_token=bearer_token,\
                       consumer_key = consumer_key,\
                       consumer_secret = consumer_secret,\
                       access_token = access_token,\
                       access_token_secret = access_token_secret)
query = 'Brooklyn Nets'
tweets = client.search_recent_tweets(query=query, max_results=100)
tweet_list = []
for tweet in tweets.data:
    tweet_list.append(tweet.text)
tweet_list = pd.DataFrame(tweet_list)
tweet_list


,0
0,RT @NetsKingdomAJ: Pay Homage to a Brooklyn Ne...
1,RT @NutsAndBoltsSP: It was supposed to be Fina...
2,RT @NutsAndBoltsSP: It was supposed to be Fina...
3,It was supposed to be Finals our bust for the ...
4,By: Christopher Jordan\n\nWhat's next for the ...
...,...
95,RT @Nick_Fay_: NEW BROOKLYN BUZZ⚫️⚪️\n\n@JacMa...
96,The Brooklyn Nets have hired Ime Udoka to be t...
97,RT @EthosNets: With the season now (sadly) in ...
98,RT @Nick_Fay_: NEW BROOKLYN BUZZ⚫️⚪️\n\n@JacMa...


In [43]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [41]:
search_words = 'brooklyn'

In [ ]:
api.search_tweets

In [53]:
tweets_list = [[tweet.text, tweet.created_at, tweet.retweet_count, tweet.favorite_count] for tweet in tweets]

Unexpected parameter: start_date


In [54]:
tweets_list

[['@libsoftiktok Why are snitching on people Chaya Raichik of Brooklyn Ny?',
  datetime.datetime(2022, 5, 2, 18, 11, 17, tzinfo=datetime.timezone.utc),
  0,
  0],
 ['@shellyduarte10 Congratulation from Brooklyn, Sir !!',
  datetime.datetime(2022, 5, 2, 18, 11, 15, tzinfo=datetime.timezone.utc),
  0,
  0],
 ['S8 of Brooklyn 99 😬',
  datetime.datetime(2022, 5, 2, 18, 11, 8, tzinfo=datetime.timezone.utc),
  0,
  0],
 ['Im going to be in Brooklyn and DC for some shows in June. Come hangout. It will be a good time.… https://t.co/fAwZrBU0Uw',
  datetime.datetime(2022, 5, 2, 18, 11, 1, tzinfo=datetime.timezone.utc),
  0,
  3],
 ['@barstoolsports This ain’t Brooklyn',
  datetime.datetime(2022, 5, 2, 18, 11, 1, tzinfo=datetime.timezone.utc),
  0,
  0]]

In [28]:
# pulling NBA game specific data

df_data = pd.read_csv("nbaData.csv").drop('Unnamed: 0', axis=1)
df_odds = pd.read_csv("nbaOdds.csv")
df_results = pd.read_csv("nbaResults.csv")

In [76]:
df_data.dateTime.iloc[36]

Timestamp('2022-04-28 22:00:00')

In [31]:
df_data.dateTime = pd.to_datetime(df_data.dateTime)

In [77]:
# running sentiment analysis for 1 game

df_data.iloc[36]

__id__                  UTADAL4282022
awayTeam             Dallas Mavericks
gameState                       Final
homeTeam                    Utah Jazz
id                      UTADAL4282022
sbId                            18043
awayDiff                          2.0
awayOutcome                         W
dateTime          2022-04-28 22:00:00
homeDiff                         -2.0
homeOutcome                         L
score                           96-98
sport                             nba
total                           194.0
awayOdds                         -105
awaySpread                       -1.0
awaySpreadOdds                    100
homeOdds                          105
homeSpread                        1.0
homeSpreadOdds                    100
over                            210.5
overOdds                          100
under                           210.5
underOdds                         100
Name: 36, dtype: object

In [33]:
# import pytz
# from datetime import datetime

# game_datetime_end = datetime(game_datetime_end, tzinfo='US/Eastern')

In [86]:
home_team = df_data.iloc[36]['homeTeam']
away_team = df_data.iloc[36]['awayTeam']

game_datetime = df_data.iloc[36]['dateTime']

## Accounting for UTC conversion requred in tweet search

game_datetime_end = game_datetime - timedelta(hours=7, minutes=0)
game_datetime_start = game_datetime - timedelta(hours=12, minutes=0)

In [82]:
# game_datetime_end

# tweets = tweepy.Cursor(api.search_tweets,
#               q=search_words,
#               #since=game_datetime_start, 
#               until=game_datetime_end,
#               lang="en").items(5)

# tweets_list = [[tweet.text, tweet.created_at, tweet.retweet_count, tweet.favorite_count] for tweet in tweets]

# tweets_list

In [89]:
game_datetime_end

Timestamp('2022-04-28 15:00:00')

In [90]:
game_datetime_start

Timestamp('2022-04-28 10:00:00')

In [87]:
home_query = home_team
tweets = client.search_recent_tweets(query=home_query, max_results=100, start_time=game_datetime_start, end_time=game_datetime_end)
tweet_list = []
for tweet in tweets.data:
    tweet_list.append(tweet.text)
tweet_list = pd.DataFrame(tweet_list)
tweet_list

,0
0,RT @Ratiothen22: Go on google and type Utah ja...
1,El equipo los necesita hoy. \n\nPor favor no n...
2,RT @All_Things_Mavs: this is literally what po...
3,2do regalo del día gente volvimos ala nba. Don...
4,RT @dalton_trigg: Google “Utah Jazz Owner.” Yo...
...,...
95,RT @dalton_trigg: Google “Utah Jazz Owner.” Yo...
96,RT @dalton_trigg: Google “Utah Jazz Owner.” Yo...
97,RT @dalton_trigg: Google “Utah Jazz Owner.” Yo...
98,https://t.co/HnNpfo9L52 Dallas Mavericks vs 22...


In [88]:
for item in tweet_list[0]:
    print (item)

RT @Ratiothen22: Go on google and type Utah jazz owner 😂
El equipo los necesita hoy. 

Por favor no nos decepcionen. 

¡VAMOS EL MONTAÑOSO UTAH JAZZ! #TakeNote https://t.co/ipSnsfThr3
RT @All_Things_Mavs: this is literally what pops up when you Google “Utah Jazz Owner”

search it right, see for yourself and reply with a s…
2do regalo del día gente volvimos ala nba. Dond la kasa va a defender nuestra plata el día d hoy a forzar el 7m juego
Utah jazz a ganar  defiende nuestro 2do regalo gente
Delen RT https://t.co/rw6ZtOkzcf
RT @dalton_trigg: Google “Utah Jazz Owner.” You’re welcome.
No way google got jalen brunson as the utah jazz owner
Dallas Mavericks vs. Utah Jazz - 4/28/22 Free Pick &amp; NBA Betting Prediction https://t.co/ruNzPiiMOb
.
#Betting #FreePicks https://t.co/Y2QU7IIpKk
RT @dalton_trigg: Google “Utah Jazz Owner.” You’re welcome.
RT @All_Things_Mavs: this is literally what pops up when you Google “Utah Jazz Owner”

search it right, see for yourself and reply with a s…
RT @d